# SD-TSIA 211 -  Let’s reverse-engineer the data center

BENEDETTI DA ROSA Giovanni and CRISTIAN CHÁVEZ Alejandro

Question 3.1
Show that if $Aw = b$, then $y(t) = \frac{w_1^T \tilde{x}(t) + w_0}{w_2^T \tilde{x} (t) + 1}$

$$(Aw)_t = \tilde{x}(t)^T w_1 + w_0  - y(t) \times \tilde{x}(t)^T w_2$$

$$(y)_t = b_t$$

If $Aw = b \implies (Aw)_t = (y)_t = b_t $

So $$(y)_t = \tilde{x}(t)^T w_1  + w_0  - y(t) \times \tilde{x}(t)^T w_2   \iff  \tilde{x}(t)^Tw_2  + w_{0} = y(t) \times \tilde{x}(t)^{T} w_{2} + y(t) \iff $$

$$y(t) = \frac{ \tilde{x}(t)^T w_1  + w_0}{ \tilde{x}(t)^T w_2 + 1}$$


Rearranging the terms and remembering that the inner product is symmetric: $\langle b,a \rangle = \langle a,b \rangle = a^Tb = ab^T$, we have:
 $$y(t) = \frac{w_1^T \tilde{x}(t) + w_0}{w_2^T \tilde{x} (t) + 1}$$




In [39]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]

Question 3.2 Solve this least squares problem using the function ```numpy.linalg.lstsq```

In [40]:
coefs = np.linalg.lstsq(A, b, rcond=None)[0]

Question 3.3
Evaluate the quality of the solution found on the test set.

In [41]:
prediction = A_test@coefs

print(f"Mean error: {np.mean((b_test - prediction)**2)}")

Mean error: 780.8984793523564


Question 3.4:

In order to improve the generalization power of the model, we consider a 2 regularization :

$$\min_w \frac{1}{2}||Aw - b||^2 + \frac{\lambda }{2}||w||^2$$
Where $\lambda=100$. Solve the problem and compare the test mean square error with the unregularized one.




$$\nabla f1 = A^T(Aw-b) + \lambda w = 0 \implies A^TAw-A^Tb + \lambda w = 0 \implies w = (A^TA+\lambda I)^{-1} A^Tb$$

In [42]:

lambda_value  = 100
term1 = np.linalg.inv(A.T@A + lambda_value * np.identity(A.shape[1]))
term2 = A.T@b
W = term1@term2

prediction = A_test@W
print(f"Mean error: {np.mean((b_test - prediction)**2)}")

Mean error: 301.05482809203016


Question 3.5
Calculate the gradient of $f1 : w→\min_w \frac{1}{2}||Aw - b||^2 + \frac{\lambda }{2}||w||^2$.  Is the function convex ?

As this is a function $\mathbb{R}^n→\mathbb{R}$, we can express the gradient using the following expression:
$$f(x + h) =  f(x) + \langle ∇f(x), h \rangle + o(h) $$
Applying to this function we obtain:
$$f(w + h) = \frac{1}{2}||A(w+h) - b||^2 + \frac{\lambda }{2}||w+h||^2 = f(w) +  \langle Aw-b, Ah \rangle + \lambda \langle w, h \rangle + o(h) $$

But: $$ \langle Aw-b, Ah \rangle = hA^T(Aw-b) = \langle A^T(Aw-b), h \rangle$$

So we can rewrite this, using the linearity property of the inner product $\langle u, w \rangle + \langle v, w \rangle =  \langle u +v, w \rangle$:

$$f(w + h) = f(w) + \langle A^T(Aw-b), h \rangle +  \langle \lambda w, h \rangle + o(h) \iff \nabla f =  A^T(Aw-b) + \lambda w $$


To prove that this function is convex, we will show that the Hessian is positive semi-definite. We will do that by finding the Jabobian of the gradient function.

We recall that the gradient is a function g: $\mathbb{R}^n→\mathbb{R}^m$ function and use the following expression:

$$g(x + h) = g(x) + J_g(x)h + o(h)$$

Applying this:

$$g(w + h) =  A^T(A(w+h)-b) + \lambda (w+h) =  g(x) +  A^TAh + \lambda I h + o(h) \iff J_g(w) =  A^TA + \lambda I$$

So, suposing $\lambda > 0 $,  $\lambda I$ is positive definite.

Also, we know $A \in \mathbb{R}^{m \times n} A^TA$ is always positive semi-definite.


As the Hessian is the sum of a positive semi-definite matrix and a positive definite matrix the result is a positive definite matrix ($A^TA + \lambda I$), thus $f_1$ is  convex.

Actually, looking to the formula of the function we can conclude that is strong convex, once $\frac{1}{2}||Aw - b||^2$ is a convex function that maintains it convexity after been subtracted by $\frac{\lambda }{2}||w||^2(\frac{\mu}{2}||w||^2)$.




Question 3.6
Implement gradient descent to minimize f1. What step size are you choosing ? How many
iterations are needed to get wk such that $k∇f(wk)k ≤ 1$ ?

Now we will see the optimized step for this specific function:

$$|\nabla f(w_1) - \nabla f(w_2)| = ||A^T(Aw_1-b) + \lambda w_1 - A^T(Aw_2-b) + \lambda w_2|| = ||A^TA(w_1 - w_2) + \lambda (w_1 - w_2)|| \implies $$

$$||(A^TA+ \lambda)(w_1 - w_2)|| \leq ||A^TA + I \lambda||_{op}(w_1 - w_2)$$

As $A^TA + I \lambda$ is symetric, $$||A^TA + I \lambda||_{op}$$ is the square root of the largest eigenvalue($\lambda_{m}$).

Therefore: 

$$||\nabla f(w_1) - \nabla f(w_2)|| \leq \sqrt{\lambda_{m}} ||w_1 - w_2|| $$ 

Where $L = \sqrt{\lambda_{m}}$ and it represents the *L-lipschitz* constant.

We use as an optimal step the rate $\frac{2}{L+ \mu}$, where $\mu$ is the strong-convexity constant, as described in page 21 of the poly.


In [29]:
def gradient_descent(A, b, lambda_val, learning_rate=1e-3):
    # Initialize variables
    max_iterations = 10000000
    epsilon = 1
    
    w = np.zeros(A.shape[1])
    
    for i in range(max_iterations):
    
        # Calculate gradient
        gradient = A.T @ (A @ w - b) + lambda_val * w
        
        # Update w
        w -= learning_rate * gradient
        
        
        # Check convergence
        if np.linalg.norm(gradient) <= epsilon:
            print(f"Number of iterations: {i}")
            print(f"Norm of grad: {np.linalg.norm(gradient)}")
            break
    
    return w

In [30]:
lambda_val = 100

def operator_norm(matrix):
    singular_values = np.linalg.svd(matrix, compute_uv=False)
    return np.max(singular_values)

matrix =  A.T @ A + np.eye(A.shape[1]) * lambda_val
max_eigenvalue = operator_norm(matrix)
print(max_eigenvalue)

w = gradient_descent(A, b, lambda_val, learning_rate = (2/(max_eigenvalue+lambda_val)))

3492643.105358824
Number of iterations: 201792
Norm of grad: 0.9999632975841187


$$\min_w \frac{1}{2}||Aw - b||^2 + \lambda ||w||_1$$

Question 4.1
Write the objective function as F2 = f2 + g2 where f2 is differentiable and the proximal operator of g2 is easy to compute. Recall the formula for proxg2. Calculate the gradient of f2

$$ p_i = prox_{\lambda g2}(y) = arg min_w |w| + \frac{1}{2 \lambda}||w-y||^2 \Longleftrightarrow $$

By Fermat's Rule:


$$0 \in \partial(|.| +  \frac{1}{2 \lambda}(|.-w_i|^2)(p_i)\Longleftrightarrow  $$
$$ 0 \in \partial |.|(p_i) +   \frac{1}{\lambda}(p_i-w_i)\Longleftrightarrow \frac{1}{\lambda}(w_i - p_i) \in \partial \lambda|.|(w) \Longleftrightarrow $$


In the component i of the vector $w_i$:
$$ (w_i - p_i) \in 

\begin{align*}
\Biggl\{
\begin{aligned}
-\lambda, p_i<0\\
[-\lambda,\lambda], p_i=0\\
\lambda, p_i>0
\end{aligned}
\biggr.
\end{align*}  \Longleftrightarrow  

p_i \in

\begin{align*}
\Biggl\{
\begin{aligned}
w_i + \lambda, p_i<0\\
w_i + [-\lambda,\lambda], p_i=0\\
w_i - \lambda, p_i>0
\end{aligned}
\biggr.
\end{align*}$$

If $p_i <0: p_i = w_i + \lambda < 0 \implies x_i <- \lambda$

If $p_i >0: p_i = w_i - \lambda > 0 \implies x_i > \lambda$

If $p_i =0 \implies p_i \in w_i + [-\lambda,\lambda]  \Longleftrightarrow$
$$ 
p_i \in
\begin{align*}
\Biggl\{
\begin{aligned}
w_i  + \lambda,  w_i <- \lambda\\
0, |w_i| \leq \lambda\\
w_ i - \lambda,  w_i > \lambda
\end{aligned}
\biggr.
\end{align*}  \implies [|w_i| - \lambda]^+ sign(w_i), s.t [z]^+  = max[z,0]$$

So: $p = prox_g(w) = p_{i : 1 \leq i \leq n}  =(prox(w_i)_{\lambda |.|})_{i : 1 \leq i \leq n} = ([|w_i| - \lambda]^+ sign(w_i))_{i : 1 \leq i \leq n} = [|w_i| - \lambda]^+ \otimes sign(w_i)$


And the gradient of f2, done in previous questions

$$\nabla||A(w+h) - b||^2  = A^T(Aw-b)$$

Question 4.2 

Code the proximal gradient method. Here, we will take λ = 200. What stopping test do you
suggest ?

In [44]:
import numpy as np

def soft_thresholding(x, threshold):
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

def proximal_gradient_method(A, b, lambda_val, max_iter=1000000, tolerance=1e-5):
    
    n = A.shape[1]
    w = np.random.randn(A.shape[1])
    step_size = 1 / np.linalg.norm(A.T @ A)  # Choose an appropriate step size

    for iteration in range(max_iter):
        gradient = A.T @ (A @ w - b)
        w_new = soft_thresholding(w - step_size * gradient, lambda_val * step_size)

        if np.linalg.norm(w_new - w) < tolerance:
            break

        w = w_new
        result = w
        print("Current value:", np.linalg.norm(A @ result - b) ** 2 + lambda_val * np.linalg.norm(result, ord=1))

    return w

lambda_val = 200
result = proximal_gradient_method(A, b, lambda_val)
print("Optimal solution:", result)
print("Optimal value:", np.linalg.norm(A @ result - b) ** 2 + lambda_val * np.linalg.norm(result, ord=1))

Current value: 13387295.765801746
Current value: 9794120.947863173
Current value: 7853814.748571215
Current value: 6684217.05237309
Current value: 5913785.063217959
Current value: 5368978.561191287
Current value: 4961933.062723444
Current value: 4644567.073479318
Current value: 4388728.247906437
Current value: 4176898.1777114486
Current value: 3997676.911441915
Current value: 3843339.9039376494
Current value: 3708475.1384336385
Current value: 3589167.566313455
Current value: 3482520.205763789
Current value: 3386339.446393426
Current value: 3298930.053372206
Current value: 3218973.9685376184
Current value: 3145391.9462005408
Current value: 3077323.250318203
Current value: 3014057.3903976963
Current value: 2955010.1936492287
Current value: 2899686.151367499
Current value: 2847671.429316099
Current value: 2798616.12554079
Current value: 2752227.583938719
Current value: 2708237.0850134115
Current value: 2666421.325880252
Current value: 2626584.8580985703
Current value: 2588556.678012161
Cu

Question 4.3
We may try to accelerate the algorithm using line search. Compare the speed of the algorithm with fixed step size and with line search.

In [45]:
import numpy as np

def soft_thresholding(x, threshold):
    """Soft thresholding operator."""
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

def f(A, b, lambda_val, w):
    """Objective function for the proximal method."""
    return 0.5 * np.linalg.norm(A @ w - b)**2 + lambda_val * np.sum(np.abs(w))

def gradient_f(A, b, w):
    """Gradient of the objective function for the proximal method."""
    return A.T @ (A @ w - b)

def proximal_gradient_method_with_line_search(A, b, lambda_val, max_iter=90000, tolerance=1e-5, h=0.1):
    """Proximal gradient method with line search."""
    n = A.shape[1]
    w = np.zeros(n)  # Initial guess

    for iteration in range(max_iter):
        grad = gradient_f(A, b, w)

        # Line search to find the step size gamma
        gamma = 1.0
        while f(A, b, lambda_val, soft_thresholding(w - gamma * grad, lambda_val * gamma)) > f(A, b, lambda_val, w) + np.dot(grad, soft_thresholding(w - gamma * grad, lambda_val * gamma) - w) + (1 / (2 * gamma)) * np.linalg.norm(w-soft_thresholding(w - gamma * grad, lambda_val * gamma))**2:
            gamma *= 0.5

        # Update the iterate
        w_new = soft_thresholding(w - gamma * grad, lambda_val * gamma)

        # Check for convergence
        #if np.linalg.norm(w_new - w) < tolerance:
            #break

        w = w_new
        print("Current value:", f(A, b, lambda_val, w))

    return w

# Example usage:
# Assuming A and b are defined elsewhere, and lambda_val is 200
lambda_val = 200
result = proximal_gradient_method_with_line_search(A, b, lambda_val)
print("Optimal solution:", result)
print("Optimal value:", f(A, b, lambda_val, result))


Current value: 5388.066646310514
Current value: 5069.294457410322
Current value: 4880.870710624711
Current value: 4833.012549959366
Current value: 4769.414662734414
Current value: 4723.55181631622
Current value: 4705.12968947598
Current value: 4661.97426235725
Current value: 4624.839368291555
Current value: 4618.1488013001435
Current value: 4612.74101080497
Current value: 4603.061320749619
Current value: 4567.8328077109945
Current value: 4505.305577762062
Current value: 4504.64504484636
Current value: 4504.000853900885
Current value: 4503.92188133219
Current value: 4503.882489442385
Current value: 4503.843151032395
Current value: 4503.823505112022
Current value: 4503.813687949234
Current value: 4503.803874092185
Current value: 4503.798968608173
Current value: 4503.796516226929
Current value: 4503.795290126449
Current value: 4503.794064077469
Current value: 4503.793451075504
Current value: 4503.793144580153
Current value: 4503.793067956967
Current value: 4503.792991333985
Current value:

KeyboardInterrupt: 